In [ ]:
from torchvision.datasets import ImageFolder
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import random_split

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])

In [ ]:
dataset = ImageFolder('Spec',transform=data_transform)

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Split the dataset into train and test subsets
train_set, test_set = random_split(dataset, [train_size, test_size])

In [ ]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

In [ ]:
classes = dataset.classes
num_emotions = len(classes)

In [ ]:
classes

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad']

In [ ]:
model = models.resnet50(pretrained=True)
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
num_features = model.fc.in_features     #extract fc layers features
model.fc = nn.Linear(num_features, num_emotions)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad and 'fc' not in name:
        print(name)
        param.requires_grad = False

conv1.weight
bn1.weight
bn1.bias
layer1.0.conv1.weight
layer1.0.bn1.weight
layer1.0.bn1.bias
layer1.0.conv2.weight
layer1.0.bn2.weight
layer1.0.bn2.bias
layer1.0.conv3.weight
layer1.0.bn3.weight
layer1.0.bn3.bias
layer1.0.downsample.0.weight
layer1.0.downsample.1.weight
layer1.0.downsample.1.bias
layer1.1.conv1.weight
layer1.1.bn1.weight
layer1.1.bn1.bias
layer1.1.conv2.weight
layer1.1.bn2.weight
layer1.1.bn2.bias
layer1.1.conv3.weight
layer1.1.bn3.weight
layer1.1.bn3.bias
layer1.2.conv1.weight
layer1.2.bn1.weight
layer1.2.bn1.bias
layer1.2.conv2.weight
layer1.2.bn2.weight
layer1.2.bn2.bias
layer1.2.conv3.weight
layer1.2.bn3.weight
layer1.2.bn3.bias
layer2.0.conv1.weight
layer2.0.bn1.weight
layer2.0.bn1.bias
layer2.0.conv2.weight
layer2.0.bn2.weight
layer2.0.bn2.bias
layer2.0.conv3.weight
layer2.0.bn3.weight
layer2.0.bn3.bias
layer2.0.downsample.0.weight
layer2.0.downsample.1.weight
layer2.0.downsample.1.bias
layer2.1.conv1.weight
layer2.1.bn1.weight
layer2.1.bn1.bias
layer2.1.conv2.we

In [ ]:
def plot_accuracy(train_accuracy,label=""):
    epochs = np.arange(1, len(train_accuracy) + 1)  # Create an array of epoch numbers

    # Plotting the accuracy
    plt.plot(epochs, train_accuracy, 'b', label=f'{label} Accuracy')
    #plt.plot(epochs, val_accuracy, 'r', label='Validation Accuracy')
    plt.title(f'{label} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

In [ ]:
def train_model():
  acc_valid = []
  acc_train = []
  num_epochs = 20   #(set no of epochs)
  start_time = time.time() #(for showing time)
  for epoch in range(num_epochs): #(loop for every epoch)
      print("Epoch {} running".format(epoch)) #(printing message)
      """ Training Phase """
      model.train()    #(training model)
      running_loss = 0.   #(set loss 0)
      running_corrects = 0
      # load a batch data of images
      with tqdm(train_loader,unit="batch") as tepoch:
          tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
          for inputs, labels in tepoch:
              inputs = inputs.to(device)
              labels = labels.to(device)
              # forward inputs and get output
              optimizer.zero_grad()
              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)
              # get loss value and update the network weights
              loss.backward()
              optimizer.step()
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)
              tepoch.set_postfix(loss=loss.item())
      # epoch_loss = running_loss / len(train_data)
      # epoch_acc = running_corrects / len(train_data) * 100.
      # acc_train.append(epoch_acc)
      #print('[Train #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() -start_time))

      with torch.no_grad():
          running_loss = 0.
          running_corrects = 0
          for inputs, labels in test_loader:
              inputs = inputs.to(device)
              labels = labels.to(device)
              outputs = model(inputs)
              _, preds = torch.max(outputs, 1)
              loss = criterion(outputs, labels)
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

          epoch_loss = running_loss / len(test_set)
          epoch_acc = running_corrects / len(test_set) * 100.
          acc_valid.append(epoch_acc)
          print('[Valid #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time()- start_time))



In [ ]:
train_model()

Epoch 0 running


Epoch 1/20: 100%|██████████| 123/123 [00:38<00:00,  3.21batch/s, loss=1.38]


[Valid #0] Loss: 0.9983 Acc: 66.6327% Time: 48.3460s
Epoch 1 running


Epoch 2/20: 100%|██████████| 123/123 [00:39<00:00,  3.09batch/s, loss=0.932]


[Valid #1] Loss: 0.9169 Acc: 67.1429% Time: 96.9011s
Epoch 2 running


...

[Valid #17] Loss: 0.5222 Acc: 82.1429% Time: 880.9158s
Epoch 18 running


Epoch 19/20: 100%|██████████| 123/123 [00:39<00:00,  3.13batch/s, loss=0.899]


[Valid #18] Loss: 0.5155 Acc: 83.7755% Time: 930.1695s
Epoch 19 running


Epoch 20/20: 100%|██████████| 123/123 [00:42<00:00,  2.90batch/s, loss=0.621]


[Valid #19] Loss: 0.5036 Acc: 83.5714% Time: 982.0368s


In [ ]:
model_save_name = 'SER_Teess_Savee_Ravdess_classifier.pth'
path = f"/content/drive/MyDrive/{model_save_name}"
torch.save(model.state_dict(), path)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')